In [1]:
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
url = f"https://api.opendota.com/api/teams"

In [3]:
response = requests.get(url)
teams_data = response.json()

In [4]:
heroic_id = []
for teams in teams_data: 
    if teams['name'] == 'HEROIC': 
        heroic_id = teams['team_id']
        print(heroic_id)

9303484


In [5]:
def team_info(heroic_id):
    url = f"https://api.opendota.com/api/teams/{heroic_id}"
    response = requests.get(url)
    return response.json()
teste = team_info(heroic_id)

In [6]:
def win_rate(heroic_id):
    team_data = team_info(heroic_id)
    win = team_data['wins']
    losses = team_data['losses']
    total = win + losses
    win_rate = (win / total) * 100 if total > 0 else 0
    return round(win_rate, 1)
print(f"A Heroic tem {win_rate(heroic_id)}% de WinRate")

A Heroic tem 54.0% de WinRate


In [7]:
def get_matches(heroic_id):
    url = f"https://api.opendota.com/api/teams/{heroic_id}/matches"
    response = requests.get(url)
    matches = response.json()
    return matches[:4]

In [8]:
matches = get_matches(heroic_id)

In [9]:
def format_duration(duration):
    hours = duration // 3600             
    minutes = (duration % 3600) // 60    
    seconds = duration % 60    
    if hours > 0:
        return f"{hours}h {minutes}m {seconds}s"
    else:
        return f"{minutes}m {seconds}s"

In [10]:
def team_side(match):
    if match['radiant']:
        return True  
    else:
        return False

In [11]:
for match in matches:
    is_radiant = team_side(match)
    if (is_radiant and match['radiant_win']) or (not is_radiant and not match['radiant_win']):
        resultado = 'Vitória'
    else:
        resultado = 'Derrota'
    duration_formatted = format_duration(match['duration'])
    print(f"Resultado: {resultado}, Duração: {duration_formatted}")

Resultado: Derrota, Duração: 49m 54s
Resultado: Vitória, Duração: 34m 38s
Resultado: Derrota, Duração: 1h 6m 37s
Resultado: Derrota, Duração: 26m 37s


In [12]:
match_ids = []
for match in matches:
    match_ids.append(match['match_id']) 
match_ids

[8011750772, 8011674211, 8011300761, 8011202507]

In [13]:
count = 0
count1 = 1
total_kils = 0

for match in matches:
    is_radiant = team_side(match)
    radiant_score = match['radiant_score']
    dire_score = match['dire_score']
    total_score_match = radiant_score + dire_score
    total_kils += total_score_match 
    print(f"A partida {count1} teve {total_score_match} abates")
    if is_radiant == True:
        print(f"A Heroic teve {radiant_score} kills")
        print(f"O outro time teve {dire_score} kills")
    
    elif is_radiant == False:
        print(f"A Heroic teve {dire_score} kills")
        print(f"O outro time teve {radiant_score} kills")

    count += 1
    count1 += 1
    
media = (total_kils/count)
print(f"Media de kils nas ultimas {count} partidas foi de {media} kils")
print(f"Total de kills em todas as partidas: {total_kils}")

A partida 1 teve 127 abates
A Heroic teve 56 kills
O outro time teve 71 kills
A partida 2 teve 52 abates
A Heroic teve 27 kills
O outro time teve 25 kills
A partida 3 teve 91 abates
A Heroic teve 45 kills
O outro time teve 46 kills
A partida 4 teve 45 abates
A Heroic teve 15 kills
O outro time teve 30 kills
Media de kils nas ultimas 4 partidas foi de 78.75 kils
Total de kills em todas as partidas: 315


In [14]:
total_duration = 0
for match in matches:
    total_duration += match['duration']  
media = total_duration / 4
print(f"O tempo total de duracao das partidas foi de {format_duration(total_duration)}")  
print(f"A media de tempo de cada partida foi de {format_duration(media)}")  

O tempo total de duracao das partidas foi de 2h 57m 46s
A media de tempo de cada partida foi de 44.0m 26.5s


In [15]:
def get_match_detail(match_ids):
    url = f"https://api.opendota.com/api/matches/{match_id}"
    response = requests.get(url)
    return response.json()

In [23]:
url = f'https://api.opendota.com/api/teams/{heroic_id}/players'
response = requests.get(url) 
players1 = response.json()
players_name = []
for player in players1:
    account_id = player.get('name')
    in_team = player.get("is_current_team_member")
    if account_id and in_team == True:
        players_name.append(account_id)

In [22]:
kda_acumulado = {player_name: [] for player_name in players_name} 
for match_id in match_ids:
    match_data = get_match_detail(match_id)
    players = match_data['players']
    
    for player in players:
        jogador_nome = player.get('name', 'Desconhecido')
        if jogador_nome in players_name:
            kills = player['kills']
            deaths = player['deaths']
            assists = player['assists']
            kda = (kills + assists) / (deaths if deaths > 0 else 1) 
            kda_acumulado[jogador_nome].append(kda)

for jogador_nome, kda_lista in kda_acumulado.items():
    if kda_lista:
        kda_medio = sum(kda_lista) / len(kda_lista)
        print(f"O jogador {jogador_nome} tem uma média de {kda_medio:.2f} de KDA.")
    else:
        print(f"O jogador {jogador_nome} não possui partidas registradas.")

O jogador KingJungles tem uma média de 1.81 de KDA.
O jogador 4nalog <01 tem uma média de 7.31 de KDA.
O jogador Davai Lama tem uma média de 3.97 de KDA.
O jogador Scofield tem uma média de 2.22 de KDA.
O jogador k1 hector tqmM não possui partidas registradas.
O jogador ɹǝʞɹɐd⚔️ tem uma média de 7.39 de KDA.
